<a href="https://colab.research.google.com/github/act-moises/diplomado/blob/main/C%C3%B3digo_Pr%C3%A1ctica_5_ModV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from scipy.stats import chisquare
from scipy.stats import kstest
from scipy.stats import anderson
from scipy.stats import norm
from scipy.stats import chi2
from scipy.stats import t

In [20]:

# Cargar el archivo CSV (asegúrate de subir el archivo a Google Colab)
file_path = 'data.csv'  # Cambia esto con el nombre del archivo subido
df = pd.read_csv(file_path)

# Excluir la columna 'N.' y seleccionar solo las columnas numéricas restantes
df = df.drop(columns=['N.'], errors='ignore')
numeric_cols = df.select_dtypes(include='number')


# Pruebas de uniformidad

## J-Quadrada

In [21]:
# Realizar la prueba de ji cuadrada en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    observed_frequencies = pd.cut(numeric_cols[col], bins=10).value_counts().values
    expected_frequencies = [len(numeric_cols[col]) / 10] * 10
    chi2_stat, p_value = chisquare(observed_frequencies, f_exp=expected_frequencies)

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no es uniforme)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (es uniforme)'

    results[col] = {
        'Chi2 Statistic': chi2_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Chi2 Statistic   P-value  \
Método Cuadrados medios                     23.2  0.005762   
Método Congruencial                         22.2  0.008266   
Método Congruencial  Mixto                   2.0  0.991468   
Aleatorios Google Sheets {RAND()}            6.2  0.719747   

                                                                      Conclusión  
Método Cuadrados medios               Rechaza la hipótesis nula (no es uniforme)  
Método Congruencial                   Rechaza la hipótesis nula (no es uniforme)  
Método Congruencial  Mixto         No se rechaza la hipótesis nula (es uniforme)  
Aleatorios Google Sheets {RAND()}  No se rechaza la hipótesis nula (es uniforme)  


## Kolmogorov-Smirnov

In [22]:
# Realizar la prueba de Kolmogorov-Smirnov en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Realizar la prueba de Kolmogorov-Smirnov
    ks_stat, p_value = kstest(numeric_cols[col], 'uniform', args=(0, 1))

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no es uniforme)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (es uniforme)'

    results[col] = {
        'KS Statistic': ks_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  KS Statistic   P-value  \
Método Cuadrados medios                 0.0874  0.406355   
Método Congruencial                    0.09322  0.329248   
Método Congruencial  Mixto             0.04429  0.984671   
Aleatorios Google Sheets {RAND()}      0.08091  0.503744   

                                                                      Conclusión  
Método Cuadrados medios            No se rechaza la hipótesis nula (es uniforme)  
Método Congruencial                No se rechaza la hipótesis nula (es uniforme)  
Método Congruencial  Mixto         No se rechaza la hipótesis nula (es uniforme)  
Aleatorios Google Sheets {RAND()}  No se rechaza la hipótesis nula (es uniforme)  


## Aoderson-Ddling

In [23]:
# Normalizar los datos entre 0 y 1 para la prueba de Anderson-Darling
results = {}

for col in numeric_cols.columns:
    # Normalizar la columna entre 0 y 1
    col_data = (numeric_cols[col] - numeric_cols[col].min()) / (numeric_cols[col].max() - numeric_cols[col].min())

    # Realizar la prueba de Anderson-Darling para la distribución normal
    ad_result = anderson(col_data, dist='norm')

    # Evaluar la hipótesis nula usando el umbral más cercano
    stat_value = ad_result.statistic
    critical_value = ad_result.critical_values[2]  # Usando el nivel de significancia del 5%

    # Verificar si se rechaza la hipótesis nula
    if stat_value > critical_value:
        conclusion = 'Rechaza la hipótesis nula (no es uniforme)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (es uniforme)'

    results[col] = {
        'AD Statistic': stat_value,
        'Critical Value (al 5%)': critical_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  AD Statistic Critical Value (al 5%)  \
Método Cuadrados medios               1.588644                  0.759   
Método Congruencial                   2.776679                  0.759   
Método Congruencial  Mixto            1.229865                  0.759   
Aleatorios Google Sheets {RAND()}     1.964456                  0.759   

                                                                   Conclusión  
Método Cuadrados medios            Rechaza la hipótesis nula (no es uniforme)  
Método Congruencial                Rechaza la hipótesis nula (no es uniforme)  
Método Congruencial  Mixto         Rechaza la hipótesis nula (no es uniforme)  
Aleatorios Google Sheets {RAND()}  Rechaza la hipótesis nula (no es uniforme)  


# Pruebas de independencia

In [24]:
# Función para la prueba de corridas arriba y abajo
def runs_test(data):
    # Calcular las corridas (arriba y abajo)
    runs, n1, n2 = 0, 0, 0

    # Identificar si el valor es arriba o abajo respecto al promedio
    median = data.median()
    above = data > median

    # Contar el número de corridas y la frecuencia de valores arriba y abajo
    last = above.iloc[0]
    for value in above:
        if value != last:
            runs += 1
            last = value
        if value:
            n1 += 1
        else:
            n2 += 1

    # Calcular el estadístico Z
    N = len(data)
    expected_runs = ((2 * n1 * n2) / N) + 1
    std_runs = ((2 * n1 * n2) * (2 * n1 * n2 - N)) / (N**2 * (N - 1))**0.5
    z = (runs - expected_runs) / std_runs

    # Calcular el valor p a partir del estadístico Z
    p_value = 2 * (1 - norm.cdf(abs(z)))

    return z, p_value


In [25]:
# Realizar la prueba de corridas arriba y abajo en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico Z y el valor p
    z_stat, p_value = runs_test(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no hay independencia)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (hay independencia)'

    results[col] = {
        'Z Statistic': z_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Z Statistic   P-value  \
Método Cuadrados medios             -0.000203  0.999838   
Método Congruencial                  0.000041  0.999968   
Método Congruencial  Mixto            0.00103  0.999178   
Aleatorios Google Sheets {RAND()}    0.000284  0.999773   

                                                                          Conclusión  
Método Cuadrados medios            No se rechaza la hipótesis nula (hay independe...  
Método Congruencial                No se rechaza la hipótesis nula (hay independe...  
Método Congruencial  Mixto         No se rechaza la hipótesis nula (hay independe...  
Aleatorios Google Sheets {RAND()}  No se rechaza la hipótesis nula (hay independe...  


## Prueba de corridas arriba y debajo de la media

In [26]:
# Función para la prueba de corridas arriba y debajo de la media
def runs_test_mean(data):
    # Calcular la media
    mean = data.mean()

    # Determinar las corridas por encima y por debajo de la media
    runs, n1, n2 = 0, 0, 0
    above = data > mean

    # Contar el número de corridas y la frecuencia de valores por encima y por debajo
    last = above.iloc[0]
    for value in above:
        if value != last:
            runs += 1
            last = value
        if value:
            n1 += 1
        else:
            n2 += 1

    # Calcular el estadístico Z
    N = len(data)
    expected_runs = ((2 * n1 * n2) / N) + 1
    std_runs = ((2 * n1 * n2) * (2 * n1 * n2 - N)) / (N**2 * (N - 1))**0.5
    z = (runs - expected_runs) / std_runs

    # Calcular el valor p a partir del estadístico Z
    p_value = 2 * (1 - norm.cdf(abs(z)))

    return z, p_value

In [27]:
# Realizar la prueba de corridas arriba y debajo de la media en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico Z y el valor p
    z_stat, p_value = runs_test_mean(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no hay independencia)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (hay independencia)'

    results[col] = {
        'Z Statistic': z_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Z Statistic   P-value  \
Método Cuadrados medios             -0.000197  0.999843   
Método Congruencial                  0.000041  0.999967   
Método Congruencial  Mixto           0.000981  0.999217   
Aleatorios Google Sheets {RAND()}     0.00013  0.999896   

                                                                          Conclusión  
Método Cuadrados medios            No se rechaza la hipótesis nula (hay independe...  
Método Congruencial                No se rechaza la hipótesis nula (hay independe...  
Método Congruencial  Mixto         No se rechaza la hipótesis nula (hay independe...  
Aleatorios Google Sheets {RAND()}  No se rechaza la hipótesis nula (hay independe...  


## De series

In [28]:
# Función para la prueba de series
def series_test(data, num_bins=10):
    # Discretizar los datos en bins (intervalos) para formar pares
    binned_data = pd.cut(data, bins=num_bins, labels=False)

    # Contar las frecuencias observadas de pares consecutivos
    N = len(binned_data) - 1
    observed = pd.crosstab(binned_data[:-1], binned_data[1:])

    # Calcular las frecuencias esperadas asumiendo independencia
    expected = observed.sum(axis=1).values.reshape(-1, 1) * observed.sum(axis=0).values / N

    # Calcular el estadístico ji cuadrada
    chi2_stat = ((observed - expected) ** 2 / expected).sum().sum()

    # Grados de libertad para la prueba
    df = (num_bins - 1) * (num_bins - 1)

    # Calcular el valor p
    p_value = 1 - chi2.cdf(chi2_stat, df)

    return chi2_stat, p_value

In [29]:
# Realizar la prueba de series en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico chi2 y el valor p
    chi2_stat, p_value = series_test(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no hay independencia)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (hay independencia)'

    results[col] = {
        'Chi2 Statistic': chi2_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Chi2 Statistic P-value  \
Método Cuadrados medios               891.010101     0.0   
Método Congruencial                   891.010101     0.0   
Método Congruencial  Mixto            891.010101     0.0   
Aleatorios Google Sheets {RAND()}     891.010101     0.0   

                                                                         Conclusión  
Método Cuadrados medios            Rechaza la hipótesis nula (no hay independencia)  
Método Congruencial                Rechaza la hipótesis nula (no hay independencia)  
Método Congruencial  Mixto         Rechaza la hipótesis nula (no hay independencia)  
Aleatorios Google Sheets {RAND()}  Rechaza la hipótesis nula (no hay independencia)  


## De Póker

In [30]:
# Función para clasificar manos de póker
def classify_poker_hand(number):
    digits = list(str(number)[2:7])  # Tomar los primeros 5 dígitos decimales
    counts = pd.Series(digits).value_counts()

    if counts.max() == 5:
        return 'Five of a Kind'
    elif counts.max() == 4:
        return 'Four of a Kind'
    elif counts.max() == 3 and counts.nunique() == 2:
        return 'Full House'
    elif counts.max() == 3:
        return 'Three of a Kind'
    elif counts.max() == 2 and counts.nunique() == 3:
        return 'Two Pair'
    elif counts.max() == 2:
        return 'One Pair'
    else:
        return 'All Different'

In [31]:
# Frecuencias esperadas para cada mano en la prueba de póker
expected_frequencies = {
    'Five of a Kind': 0.0001,
    'Four of a Kind': 0.0045,
    'Full House': 0.009,
    'Three of a Kind': 0.072,
    'Two Pair': 0.108,
    'One Pair': 0.504,
    'All Different': 0.3024
}

# Función para realizar la prueba de póker
def poker_test(data):
    # Clasificar cada número en una mano de póker
    hands = data.apply(classify_poker_hand)
    observed_frequencies = hands.value_counts().reindex(expected_frequencies.keys(), fill_value=0).values

    # Calcular frecuencias esperadas en función del tamaño de la muestra
    n = len(data)
    expected_counts = [n * freq for freq in expected_frequencies.values()]

    # Calcular el estadístico ji cuadrada
    chi2_stat = ((observed_frequencies - expected_counts) ** 2 / expected_counts).sum()

    # Grados de libertad para la prueba (7 manos de póker - 1)
    df = len(expected_frequencies) - 1

    # Calcular el valor p
    p_value = 1 - chi2.cdf(chi2_stat, df)

    return chi2_stat, p_value

In [32]:
# Realizar la prueba de póker en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico chi2 y el valor p
    chi2_stat, p_value = poker_test(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no hay aleatoriedad)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (hay aleatoriedad)'

    results[col] = {
        'Chi2 Statistic': chi2_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Chi2 Statistic P-value  \
Método Cuadrados medios                87.003968     0.0   
Método Congruencial                    78.306878     0.0   
Método Congruencial  Mixto             92.691799     0.0   
Aleatorios Google Sheets {RAND()}      76.025132     0.0   

                                                                        Conclusión  
Método Cuadrados medios            Rechaza la hipótesis nula (no hay aleatoriedad)  
Método Congruencial                Rechaza la hipótesis nula (no hay aleatoriedad)  
Método Congruencial  Mixto         Rechaza la hipótesis nula (no hay aleatoriedad)  
Aleatorios Google Sheets {RAND()}  Rechaza la hipótesis nula (no hay aleatoriedad)  


## De huecos

In [33]:
# Función para la prueba de huecos
def gaps_test(data, lower_bound=0.4, upper_bound=0.6, max_gap_length=5):
    # Identificar las posiciones donde los valores están dentro del rango especificado
    in_range = (data >= lower_bound) & (data <= upper_bound)

    # Contar la longitud de los huecos entre las apariciones dentro del rango
    gap_lengths = []
    gap_count = 0

    for is_in_range in in_range:
        if is_in_range:
            if gap_count > 0:
                gap_lengths.append(min(gap_count, max_gap_length))
                gap_count = 0
        else:
            gap_count += 1

    if gap_count > 0:
        gap_lengths.append(min(gap_count, max_gap_length))

    # Contar las frecuencias observadas de cada longitud de hueco
    observed = pd.Series(gap_lengths).value_counts().reindex(range(1, max_gap_length + 1), fill_value=0).values

    # Calcular las frecuencias esperadas para cada longitud de hueco
    p_in_range = upper_bound - lower_bound
    expected = [(p_in_range * (1 - p_in_range) ** (k - 1)) * len(data) for k in range(1, max_gap_length + 1)]

    # Calcular el estadístico ji cuadrada
    chi2_stat = ((observed - expected) ** 2 / expected).sum()

    # Grados de libertad para la prueba
    df = max_gap_length - 1

    # Calcular el valor p
    p_value = 1 - chi2.cdf(chi2_stat, df)

    return chi2_stat, p_value

In [34]:
# Realizar la prueba de huecos en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico chi2 y el valor p para la prueba de huecos
    chi2_stat, p_value = gaps_test(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (no hay aleatoriedad)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (hay aleatoriedad)'

    results[col] = {
        'Chi2 Statistic': chi2_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Chi2 Statistic P-value  \
Método Cuadrados medios                43.986883     0.0   
Método Congruencial                    53.747625     0.0   
Método Congruencial  Mixto             41.756414     0.0   
Aleatorios Google Sheets {RAND()}      49.557977     0.0   

                                                                        Conclusión  
Método Cuadrados medios            Rechaza la hipótesis nula (no hay aleatoriedad)  
Método Congruencial                Rechaza la hipótesis nula (no hay aleatoriedad)  
Método Congruencial  Mixto         Rechaza la hipótesis nula (no hay aleatoriedad)  
Aleatorios Google Sheets {RAND()}  Rechaza la hipótesis nula (no hay aleatoriedad)  


# Estimaciones de una DUE

## Media

In [35]:
# Función para la prueba de DUE media
def due_mean_test(data):
    # Calcular la media de la muestra
    sample_mean = data.mean()

    # Media y desviación estándar esperadas para una distribución uniforme
    expected_mean = 0.5
    sample_std = data.std(ddof=1)

    # Tamaño de la muestra
    n = len(data)

    # Calcular el estadístico t
    t_stat = (sample_mean - expected_mean) / (sample_std / (n ** 0.5))

    # Calcular el valor p (prueba bilateral)
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n-1))

    return t_stat, p_value

In [36]:
# Realizar la prueba de DUE media en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico t y el valor p para la prueba de DUE media
    t_stat, p_value = due_mean_test(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (media diferente de 0.5)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (media igual a 0.5)'

    results[col] = {
        'T Statistic': t_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  T Statistic   P-value  \
Método Cuadrados medios              0.097114  0.922832   
Método Congruencial                  0.860607  0.391534   
Método Congruencial  Mixto          -0.510706  0.610694   
Aleatorios Google Sheets {RAND()}    0.674192  0.501761   

                                                                          Conclusión  
Método Cuadrados medios            No se rechaza la hipótesis nula (media igual a...  
Método Congruencial                No se rechaza la hipótesis nula (media igual a...  
Método Congruencial  Mixto         No se rechaza la hipótesis nula (media igual a...  
Aleatorios Google Sheets {RAND()}  No se rechaza la hipótesis nula (media igual a...  


## Varianza

In [37]:
# Función para la prueba de DUE de varianza
def due_variance_test(data):
    # Calcular la varianza de la muestra
    sample_var = data.var(ddof=1)

    # Varianza esperada para una distribución uniforme en [0, 1]
    expected_var = 1 / 12

    # Tamaño de la muestra
    n = len(data)

    # Calcular el estadístico chi cuadrada
    chi2_stat = (n - 1) * sample_var / expected_var

    # Calcular el valor p (prueba bilateral)
    p_value = 2 * min(1 - chi2.cdf(chi2_stat, df=n-1), chi2.cdf(chi2_stat, df=n-1))

    return chi2_stat, p_value

In [38]:
# Realizar la prueba de DUE de varianza en cada columna numérica
results = {}
alpha = 0.05  # Nivel de significancia

for col in numeric_cols.columns:
    # Obtener el estadístico chi2 y el valor p para la prueba de DUE de varianza
    chi2_stat, p_value = due_variance_test(numeric_cols[col])

    # Evaluar la hipótesis nula
    if p_value < alpha:
        conclusion = 'Rechaza la hipótesis nula (varianza diferente de 1/12)'
    else:
        conclusion = 'No se rechaza la hipótesis nula (varianza igual a 1/12)'

    results[col] = {
        'Chi2 Statistic': chi2_stat,
        'P-value': p_value,
        'Conclusión': conclusion
    }

# Convertir los resultados a un DataFrame para visualización
results_df = pd.DataFrame(results).T

# Mostrar los resultados
print(results_df)

                                  Chi2 Statistic   P-value  \
Método Cuadrados medios                83.652449  0.269271   
Método Congruencial                   106.422694  0.574127   
Método Congruencial  Mixto               99.2565  0.947694   
Aleatorios Google Sheets {RAND()}     110.618592  0.399566   

                                                                          Conclusión  
Método Cuadrados medios            No se rechaza la hipótesis nula (varianza igua...  
Método Congruencial                No se rechaza la hipótesis nula (varianza igua...  
Método Congruencial  Mixto         No se rechaza la hipótesis nula (varianza igua...  
Aleatorios Google Sheets {RAND()}  No se rechaza la hipótesis nula (varianza igua...  
